In [1]:
import pandas as pd
from sklearn.cluster import KMeans, SpectralClustering
import matplotlib.pyplot as plt
from matplotlib.image import imread
import pandas as pd
import seaborn as sns
from sklearn.metrics import silhouette_samples, silhouette_score

In [2]:
import numpy as np
class KMeans:

    def __init__(
            self,
            n_cluster: int,
            init_pp: bool = True,
            max_iter: int = 300,
            tolerance: float = 1e-4,
            seed: int = None):
        
        self.n_cluster = n_cluster
        self.max_iter = max_iter
        self.tolerance = tolerance
        self.init_pp = init_pp
        self.seed = seed
        self.centroid = None
        self.mse = None

    def fit(self, data: np.ndarray):
        
        self.centroid = self._init_centroid(data)
        for _ in range(self.max_iter):
            distance = self._calc_distance(data)
            cluster = self._assign_cluster(distance)
            new_centroid = self._update_centroid(data, cluster)
            diff = np.abs(self.centroid - new_centroid).mean()
            self.centroid = new_centroid

            if diff <= self.tolerance:
                break

        self.mse = calc_mse(self.centroid, cluster, data)

    def predict(self, data: np.ndarray):

        distance = self._calc_distance(data)
        cluster = self._assign_cluster(distance)
        return cluster

    def _init_centroid(self, data: np.ndarray):
        
        if self.init_pp:
            np.random.seed(self.seed)
            centroid = [int(np.random.uniform()*len(data))]
            for _ in range(1, self.n_cluster):
                dist = []
                dist = [min([np.inner(data[c]-x, data[c]-x) for c in centroid])
                        for i, x in enumerate(data)]
                dist = np.array(dist)
                dist = dist / dist.sum()
                cumdist = np.cumsum(dist)

                prob = np.random.rand()
                for i, c in enumerate(cumdist):
                    if prob > c and i not in centroid:
                        centroid.append(i)
                        break
            centroid = np.array([data[c] for c in centroid])
        else:
            np.random.seed(self.seed)
            idx = np.random.choice(range(len(data)), size=(self.n_cluster))
            centroid = data[idx]
        return centroid

    def _calc_distance(self, data: np.ndarray):
        distances = []
        for c in self.centroid:
            distance = np.mean((data - c) * (data - c), axis=1)
            distances.append(distance)

        distances = np.array(distances)
        distances = distances.T
        return distances

    def _assign_cluster(self, distance: np.ndarray):
        cluster = np.argmin(distance, axis=1)
        return cluster

    def _update_centroid(self, data: np.ndarray, cluster: np.ndarray):
        centroids = []
        for i in range(self.n_cluster):
            idx = np.where(cluster == i)
            centroid = np.mean(data[idx], axis=0)
            centroids.append(centroid)
        centroids = np.array(centroids)
        return centroids


In [3]:
def quantization_error(centroids: np.ndarray, labels: np.ndarray, data: np.ndarray) -> float:
    error = 0.0
    for i, c in enumerate(centroids):
        idx = np.where(labels == i)
        dist = np.linalg.norm(data[idx] - c)
        dist /= len(idx)
        error += dist
    error /= len(centroids)
    return error

def calc_mse(centroids: np.ndarray, labels: np.ndarray, data: np.ndarray):
    distances = []
    for i, c in enumerate(centroids):
        idx = np.where(labels == i)
        dist = np.mean((data[idx] - c)**2)
        distances.append(dist)
    return np.mean(distances)


class Particle:

    def __init__(self,
                 n_cluster: int,
                 data: np.ndarray,
                 use_kmeans: bool = False,
                 w: float = 0.9,
                 c1: float = 0.5,
                 c2: float = 0.3):
        index = np.random.choice(list(range(len(data))), n_cluster)
        self.centroids = data[index].copy()
        if use_kmeans:
            kmeans = KMeans(n_cluster=n_cluster, init_pp=False)
            kmeans.fit(data)
            self.centroids = kmeans.centroid.copy()
        self.best_position = self.centroids.copy()
        self.best_score = quantization_error(self.centroids, self._predict(data), data)
        self.best_mse = calc_mse(self.centroids, self._predict(data), data)
        self.velocity = np.zeros_like(self.centroids)
        self._w = w
        self._c1 = c1
        self._c2 = c2

    def update(self, gbest_position: np.ndarray, data: np.ndarray):

        self._update_velocity(gbest_position)
        self._update_centroids(data)

    def _update_velocity(self, gbest_position: np.ndarray):
        """Update velocity based on old value, cognitive component, and social component
        """

        v_old = self._w * self.velocity
        cognitive_component = self._c1 * np.random.random() * (self.best_position - self.centroids)
        social_component = self._c2 * np.random.random() * (gbest_position - self.centroids)
        self.velocity = v_old + cognitive_component + social_component

    def _update_centroids(self, data: np.ndarray):
        self.centroids = self.centroids + self.velocity
        new_score = quantization_error(self.centroids, self._predict(data), data)
        mse = calc_mse(self.centroids, self._predict(data), data)
        self.best_mse = min(mse, self.best_mse)
        if new_score < self.best_score:
            self.best_score = new_score
            self.best_position = self.centroids.copy()

    def _predict(self, data: np.ndarray) -> np.ndarray:
        """Predict new data's cluster using minimum distance to centroid
        """
        distance = self._calc_distance(data)
        cluster = self._assign_cluster(distance)
        return cluster

    def _calc_distance(self, data: np.ndarray) -> np.ndarray:
        """Calculate distance between data and centroids
        """
        distances = []
        for c in self.centroids:
            distance = np.sum((data - c) * (data - c), axis=1)
            distances.append(distance)

        distances = np.array(distances)
        distances = np.transpose(distances)
        return distances

    def _assign_cluster(self, distance: np.ndarray) -> np.ndarray:
        """Assign cluster to data based on minimum distance to centroids
        """
        cluster = np.argmin(distance, axis=1)
        return cluster


In [4]:
class ParticleSwarmOptimizedClustering:
    def __init__(self,
                 n_cluster: int,
                 n_particles: int,
                 data: np.ndarray,
                 hybrid: bool = True,
                 max_iter: int = 100,
                 print_debug: int = 10):
        self.n_cluster = n_cluster
        self.n_particles = n_particles
        self.data = data
        self.max_iter = max_iter
        self.particles = []
        self.hybrid = hybrid

        self.print_debug = print_debug
        self.gbest_score = np.inf
        self.gbest_centroids = None
        self.gbest_mse = np.inf
        self._init_particles()

    def _init_particles(self):
        for i in range(self.n_particles):
            particle = None
            if i == 0 and self.hybrid:
                particle = Particle(self.n_cluster, self.data, use_kmeans=True)
            else:
                particle = Particle(self.n_cluster, self.data, use_kmeans=False)
            if particle.best_score < self.gbest_score:
                self.gbest_centroids = particle.centroids.copy()
                self.gbest_score = particle.best_score
            self.particles.append(particle)
            self.gbest_mse = min(particle.best_mse, self.gbest_mse)

    def run(self):
        print('Initial global best score', self.gbest_score)
        history = []
        for i in range(self.max_iter):
            for particle in self.particles:
                particle.update(self.gbest_centroids, self.data)
                # print(i, particle.best_score, self.gbest_score)
            for particle in self.particles:
                if particle.best_score < self.gbest_score:
                    self.gbest_centroids = particle.centroids.copy()
                    self.gbest_score = particle.best_score
            history.append(self.gbest_score)
            if i % self.print_debug == 0:
                print('Iteration {:04d}/{:04d} current gbest score {:.18f}'.format(
                    i + 1, self.max_iter, self.gbest_score))
        print('Finish with gbest score {:.18f}'.format(self.gbest_score))
        return history
    
    def _calc_distance(self, data: np.ndarray):
        
        distances = []
        for c in self.centroid:
            distance = np.sum((data - c) * (data - c), axis=1)
            distances.append(distance)

        distances = np.array(distances)
        distances = distances.T
        return distances

    def predict(self, distance: np.ndarray):
        
        cluster = np.argmin(distance, axis=1)
        return cluster

In [5]:
from sklearn.metrics.pairwise import euclidean_distances

def delta(ck, cl):
    values = np.ones([len(ck), len(cl)])*10000
    
    for i in range(0, len(ck)):
        for j in range(0, len(cl)):
            values[i, j] = np.linalg.norm(ck[i]-cl[j])
            
    return np.min(values)
    
def big_delta(ci):
    values = np.zeros([len(ci), len(ci)])
    
    for i in range(0, len(ci)):
        for j in range(0, len(ci)):
            values[i, j] = np.linalg.norm(ci[i]-ci[j])
            
    return np.max(values)
    
def dunn(k_list):
    
    deltas = np.ones([len(k_list), len(k_list)])*1000000
    big_deltas = np.zeros([len(k_list), 1])
    l_range = list(range(0, len(k_list)))
    
    for k in l_range:
        for l in (l_range[0:k]+l_range[k+1:]):
            deltas[k, l] = delta(k_list[k], k_list[l])
        
        big_deltas[k] = big_delta(k_list[k])

    di = np.min(deltas)/np.max(big_deltas)
    return di

In [6]:
df=pd.read_csv('norm.csv')

In [7]:
df_req = df[['Stride Length (m)', 'Cadence(steps/min)', 'Leg Length (m)',
       'Age(years)','classs']]
X  = df_req[['Stride Length (m)', 'Cadence(steps/min)', 'Leg Length (m)',
       'Age(years)']]
labels = df.classs.values

In [8]:
from sklearn.model_selection import train_test_split
x_tr,x_tt,y_tr,y_tt = train_test_split(X,labels,test_size=.30)

In [26]:
pso = ParticleSwarmOptimizedClustering(
        n_cluster=2, n_particles=50, data=X.values, max_iter=8000, print_debug=50,hybrid = False)

C:\Users\Lenovo\Anaconda3\envs\tensorflow_env\lib\site-packages\numpy\core\fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\Lenovo\Anaconda3\envs\tensorflow_env\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [27]:
hist = pso.run()

Initial global best score 166.01021549751437
Iteration 0001/8000 current gbest score 166.010215497514366234
Iteration 0051/8000 current gbest score 156.202400655832519760
Iteration 0101/8000 current gbest score 156.201150709136925343
Iteration 0151/8000 current gbest score 156.200286909056217155
Iteration 0201/8000 current gbest score 156.200273966261562464
Iteration 0251/8000 current gbest score 156.200259015273587693
Iteration 0301/8000 current gbest score 156.200196363068869232
Iteration 0351/8000 current gbest score 156.200174113823493371
Iteration 0401/8000 current gbest score 156.200155905053918559
Iteration 0451/8000 current gbest score 156.200148572569446515
Iteration 0501/8000 current gbest score 156.200147551951374680
Iteration 0551/8000 current gbest score 156.200146953038483844
Iteration 0601/8000 current gbest score 156.200146402542912938
Iteration 0651/8000 current gbest score 156.200145860704509460
Iteration 0701/8000 current gbest score 156.200145158942859780
Iteration 

Iteration 6501/8000 current gbest score 156.200145103371966115
Iteration 6551/8000 current gbest score 156.200145103371966115
Iteration 6601/8000 current gbest score 156.200145103371966115
Iteration 6651/8000 current gbest score 156.200145103371966115
Iteration 6701/8000 current gbest score 156.200145103371966115
Iteration 6751/8000 current gbest score 156.200145103371966115
Iteration 6801/8000 current gbest score 156.200145103371966115
Iteration 6851/8000 current gbest score 156.200145103371966115
Iteration 6901/8000 current gbest score 156.200145103371966115
Iteration 6951/8000 current gbest score 156.200145103371966115
Iteration 7001/8000 current gbest score 156.200145103371966115
Iteration 7051/8000 current gbest score 156.200145103371966115
Iteration 7101/8000 current gbest score 156.200145103371966115
Iteration 7151/8000 current gbest score 156.200145103371966115
Iteration 7201/8000 current gbest score 156.200145103371966115
Iteration 7251/8000 current gbest score 156.20014510337

In [28]:
pso.gbest_centroids

array([[  0.91948828, 139.7357431 ,   0.58531874,   7.64431647],
       [  0.73710048,  88.65203445,   0.72546251,  11.86183026]])

In [29]:
dunn(pso.gbest_centroids)

0.001007138678401498

In [30]:
pso.gbest_mse

92.45534232590887

In [31]:
d = pso.predict(X.values)

In [32]:
print("Silhouette Coefficient: %0.3f"
      % silhouette_score(d.reshape(-1,1),labels, metric='euclidean'))

Silhouette Coefficient: 0.107


In [33]:
from sklearn import metrics

def purity_score(y_true, y_pred):
    # compute contingency matrix (also called confusion matrix)
    contingency_matrix = metrics.cluster.contingency_matrix(y_true, y_pred)
    # return purity
    return np.sum(np.amax(contingency_matrix, axis=0)) / np.sum(contingency_matrix) 

In [34]:
purity_score(labels,d)

0.6153846153846154

In [39]:
# PSO
for _ in range(20):
    pso_rep = ParticleSwarmOptimizedClustering(
        n_cluster=2, n_particles=10, data=X.values, hybrid=False, max_iter=2000, print_debug=2000)
    pso_rep.run()
    pso_kmeans = KMeans(n_cluster=2, init_pp=False, seed=2018)
    pso_kmeans.centroid = pso_rep.gbest_centroids.copy()
    predicted_pso_rep = pso_kmeans.predict(X.values)
    print("Silhouette Coefficient : %0.3f"
      % silhouette_score(predicted_pso_rep.reshape(-1,1),labels, metric='euclidean'))
    print('mse : %0.3f' % calc_mse(centroids=pso_rep.gbest_centroids, data=X.values, labels=predicted_pso_rep))
    print('purity : %0.3f' % purity_score(labels,predicted_pso_rep))

Initial global best score 169.1671291404785
Iteration 0001/2000 current gbest score 169.167129140478493809
Finish with gbest score 156.428280666700061374
Silhouette Coefficient : 0.166
mse : 85.958
purity : 0.654
Initial global best score 158.06431541524873
Iteration 0001/2000 current gbest score 158.064315415248728414
Finish with gbest score 156.208983877895832393
Silhouette Coefficient : 0.166
mse : 85.760
purity : 0.654
Initial global best score 163.40297722831463
Iteration 0001/2000 current gbest score 163.402977228314625791
Finish with gbest score 157.595954732389628816
Silhouette Coefficient : 0.177
mse : 87.543
purity : 0.660
Initial global best score 166.83872681834143
Iteration 0001/2000 current gbest score 166.838726818341427816
Finish with gbest score 156.829568199303992060
Silhouette Coefficient : 0.166
mse : 86.411
purity : 0.654
Initial global best score 162.34414051324637
Iteration 0001/2000 current gbest score 162.344140513246372848
Finish with gbest score 156.244226256

In [40]:
# HPSO
for _ in range(20):    
    pso_rep = ParticleSwarmOptimizedClustering(
        n_cluster=2, n_particles=10, data=X.values, hybrid=True, max_iter=2000, print_debug=2000)
    pso_rep.run()
    pso_kmeans = KMeans(n_cluster=2, init_pp=False, seed=2018)
    pso_kmeans.centroid = pso_rep.gbest_centroids.copy()
    predicted_pso_rep = pso_kmeans.predict(X.values)
    print("Silhouette Coefficient : %0.3f"
      % silhouette_score(predicted_pso_rep.reshape(-1,1),labels, metric='euclidean'))
    print('mse : %0.3f' % calc_mse(centroids=pso_rep.gbest_centroids, data=X.values, labels=predicted_pso_rep))
    print('purity : %0.3f' % purity_score(labels,predicted_pso_rep))

Initial global best score 156.2000456207785
Iteration 0001/2000 current gbest score 156.200045620778496414
Finish with gbest score 156.200045620778467992
Silhouette Coefficient : 0.166
mse : 85.751
purity : 0.654
Initial global best score 161.75294448827017
Iteration 0001/2000 current gbest score 161.752944488270173906
Finish with gbest score 156.213630626609074170
Silhouette Coefficient : 0.166
mse : 85.763
purity : 0.654
Initial global best score 161.75294448827017
Iteration 0001/2000 current gbest score 161.752944488270173906
Finish with gbest score 156.218587147831669881
Silhouette Coefficient : 0.166
mse : 85.770
purity : 0.654
Initial global best score 161.4250233638429
Iteration 0001/2000 current gbest score 161.425023363842910840
Finish with gbest score 156.284596578416113744
Silhouette Coefficient : 0.166
mse : 85.874
purity : 0.654
Initial global best score 156.2000456207785
Iteration 0001/2000 current gbest score 156.200045620778496414
Finish with gbest score 156.20004562077

C:\Users\Lenovo\Anaconda3\envs\tensorflow_env\lib\site-packages\numpy\core\fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\Lenovo\Anaconda3\envs\tensorflow_env\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Finish with gbest score 156.216951948644094728
Silhouette Coefficient : 0.166
mse : 85.775
purity : 0.654
Initial global best score 156.2000456207785
Iteration 0001/2000 current gbest score 156.200045620778496414


C:\Users\Lenovo\Anaconda3\envs\tensorflow_env\lib\site-packages\numpy\core\fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\Lenovo\Anaconda3\envs\tensorflow_env\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Finish with gbest score 156.200045620778496414
Silhouette Coefficient : 0.166
mse : 85.751
purity : 0.654
Initial global best score 156.2000456207785
Iteration 0001/2000 current gbest score 156.200045620778496414
Finish with gbest score 156.200045620778467992
Silhouette Coefficient : 0.166
mse : 85.751
purity : 0.654
Initial global best score 161.75294448827017
Iteration 0001/2000 current gbest score 161.752944488270173906
Finish with gbest score 156.208491534648942434
Silhouette Coefficient : 0.166
mse : 85.763
purity : 0.654
Initial global best score 161.75294448827017
Iteration 0001/2000 current gbest score 157.827714960375772080
Finish with gbest score 156.200280284391141095
Silhouette Coefficient : 0.166
mse : 85.751
purity : 0.654
Initial global best score 161.75294448827017
Iteration 0001/2000 current gbest score 161.752944488270173906
Finish with gbest score 156.205163132719803798
Silhouette Coefficient : 0.166
mse : 85.758
purity : 0.654
